In [45]:
import pymongo 
from pymongo import MongoClient
from pprintpp import pprint as pp

# connect to database
client = MongoClient('localhost', 27017)
db = client.commands

print "PyMongo version: ", pymongo.version
print "MongoDB version: ", db.eval("db.version()")

PyMongo version:  3.0.3
MongoDB version:  3.0.7


# Bulk Operations

In [34]:
db.items.drop()

In [35]:
command = """
var bulk = db.items.initializeUnorderedBulkOp();
bulk.insert( { _id: 1, item: "abc123", status: "A", soldQty: 5000 } );
bulk.insert( { _id: 2, item: "abc456", status: "A", soldQty: 150 } );
bulk.insert( { _id: 3, item: "abc789", status: "P", soldQty: 0 } );
ret = bulk.execute( { w: "majority", wtimeout: 5000 } );

return ret.getRawResponse()
"""

db.eval(command)

{u'nInserted': 3.0,
 u'nMatched': 0.0,
 u'nModified': 0.0,
 u'nRemoved': 0.0,
 u'nUpserted': 0.0,
 u'upserted': [],
 u'writeConcernErrors': [],
 u'writeErrors': []}

# Database Commands

https://docs.mongodb.org/manual/reference/command/

## isMaster

In [47]:
command = """
return db.isMaster()
"""

pp(db.eval(command))

{
    u'ismaster': True,
    u'localTime': datetime.datetime(2015, 10, 25, 10, 42, 9, 98000),
    u'maxBsonObjectSize': 16777216,
    u'maxMessageSizeBytes': 48000000,
    u'maxWireVersion': 3,
    u'maxWriteBatchSize': 1000,
    u'minWireVersion': 0,
    u'ok': 1.0,
}


## serverStatus

In [66]:
db.command({ "serverStatus": 1 })

{u'asserts': {u'msg': 0,
  u'regular': 0,
  u'rollovers': 0,
  u'user': 7,
  u'warning': 0},
 u'backgroundFlushing': {u'average_ms': 13.018796992481203,
  u'flushes': 266,
  u'last_finished': datetime.datetime(2015, 10, 25, 10, 54, 20, 356000),
  u'last_ms': 11,
  u'total_ms': 3463},
 u'connections': {u'available': 51186, u'current': 14, u'totalCreated': 410L},
 u'cursors': {u'clientCursors_size': 0,
  u'note': u'deprecated, use server status metrics',
  u'pinned': 0,
  u'timedOut': 0,
  u'totalNoTimeout': 0,
  u'totalOpen': 0},
 u'dur': {u'commits': 25,
  u'commitsInWriteLock': 0,
  u'compression': 0.0,
  u'earlyCommits': 0,
  u'journaledMB': 0.0,
  u'timeMs': {u'commits': 3,
   u'commitsInWriteLock': 0,
   u'dt': 0,
   u'prepLogBuffer': 0,
   u'remapPrivateView': 0,
   u'writeToDataFiles': 0,
   u'writeToJournal': 0},
  u'writeToDataFilesMB': 0.0},
 u'extra_info': {u'heap_usage_bytes': 64419520,
  u'note': u'fields vary by platform',
  u'page_faults': 80},
 u'globalLock': {u'activeCl

In [48]:
command = """
return db.serverStatus()
"""

pp(db.eval(command))

{
    u'asserts': {
        u'msg': 0,
        u'regular': 0,
        u'rollovers': 0,
        u'user': 4,
        u'warning': 0,
    },
    u'backgroundFlushing': {
        u'average_ms': 13.08695652173913,
        u'flushes': 253,
        u'last_finished': datetime.datetime(2015, 10, 25, 10, 41, 20, 349000),
        u'last_ms': 16,
        u'total_ms': 3311,
    },
    u'connections': {
        u'available': 51186,
        u'current': 14,
        u'totalCreated': 380L,
    },
    u'cursors': {
        u'clientCursors_size': 0,
        u'note': u'deprecated, use server status metrics',
        u'pinned': 0,
        u'timedOut': 0,
        u'totalNoTimeout': 0,
        u'totalOpen': 0,
    },
    u'dur': {
        u'commits': 29,
        u'commitsInWriteLock': 0,
        u'compression': 0.0,
        u'earlyCommits': 0,
        u'journaledMB': 0.0,
        u'timeMs': {
            u'commits': 2,
            u'commitsInWriteLock': 0,
            u'dt': 3010,
            u'prepLogBuffer':

## currentOp and killOp

In [58]:
command = """
return db.currentOp()
"""

ret_json = db.eval(command)
pp(ret_json)

{
    u'inprog': [
        {
            u'active': True,
            u'client': u'127.0.0.1:57630',
            u'connectionId': 380,
            u'desc': u'conn380',
            u'lockStats': {
                u'Collection': {u'acquireCount': {u'R': 1L}},
                u'Database': {u'acquireCount': {u'r': 1L}},
                u'Global': {u'acquireCount': {u'W': 1L, u'r': 3L}},
                u'MMAPV1Journal': {u'acquireCount': {u'w': 1L}},
            },
            u'locks': {u'Global': u'W', u'MMAPV1Journal': u'w'},
            u'microsecs_running': 41445L,
            u'ns': u'commands',
            u'numYields': 0,
            u'op': u'query',
            u'opid': 81078,
            u'query': {
                u'$eval': Code('\nreturn db.currentOp()\n', {}),
                u'args': [],
            },
            u'secs_running': 0,
            u'threadId': u'0x3bc2d80',
            u'waitingForLock': False,
        },
    ],
}


In [60]:
opid = ret_json['inprog'][0]['opid']
print opid

81078


In [65]:
command = """
function(opid) {
  db.killOp(opid)
  db.test_command.insert({"opid" : opid})
}
"""

db.eval(command, opid)

## Collections's Stats

In [70]:
command = """
return db.items.stats()
"""

db.eval(command)

{u'avgObjSize': 112,
 u'capped': False,
 u'count': 3,
 u'indexSizes': {u'_id_': 8176},
 u'lastExtentSize': 8192.0,
 u'nindexes': 1,
 u'ns': u'commands.items',
 u'numExtents': 1,
 u'ok': 1.0,
 u'paddingFactor': 1.0,
 u'paddingFactorNote': u'paddingFactor is unused and unmaintained in 3.0. It remains hard coded to 1.0 for compatibility only.',
 u'size': 336,
 u'storageSize': 8192,
 u'totalIndexSize': 8176,
 u'userFlags': 1}

## listDatabases

In [73]:
pp(client.admin.command({"listDatabases" : 1}))

{
    u'databases': [
        {u'empty': False, u'name': u'school', u'sizeOnDisk': 33554432.0},
        {u'empty': False, u'name': u'local', u'sizeOnDisk': 83886080.0},
        {u'empty': False, u'name': u'twitter', u'sizeOnDisk': 33554432.0},
        {u'empty': False, u'name': u'test', u'sizeOnDisk': 33554432.0},
        {u'empty': False, u'name': u'pcat2', u'sizeOnDisk': 33554432.0},
        {u'empty': False, u'name': u'commands', u'sizeOnDisk': 33554432.0},
        {u'empty': False, u'name': u'reddit', u'sizeOnDisk': 33554432.0},
        {u'empty': False, u'name': u'pcat', u'sizeOnDisk': 33554432.0},
    ],
    u'ok': 1.0,
    u'totalSize': 318767104.0,
}


## Review of Commands

|Server|Db|Collection|index|
|-|-|-|-|
|isMaster|repairDabase|create (implicit)|ensureIndex|
|serverStatus|dropDabase|drop|dropIndex|
|logout|dbStats|collStats||
|getLastError|copydb|renameCollection||
||clone|count (user)||
|||aggregate (user)||
|||mapReduce (user)||
|||findAndModify (user)||
|||geo* (user)||